In [1]:
# import datetime
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datlib.FRED import *
from datlib.plots import *
import pandas_datareader.data as web


#FRED.py
#. . . 
def bil_to_mil(series):
    return series* 10**3
# . . .
#fedProject.py
# . . .
data_codes  = {# Assets
               "Balance Sheet: Total Assets ($ Mil)": "WALCL",
               "Balance Sheet Securities, Prem-Disc, Repos, and Loans ($ Mil)": "WSRLL",
               "Balance Sheet: Securities Held Outright ($ Mil)": "WSHOSHO",
               ### breakdown of securities holdings ###
               "Balance Sheet: U.S. Treasuries Held Outright ($ Mil)":"WSHOTSL",
               "Balance Sheet: Federal Agency Debt Securities ($ Mil)" : "WSHOFADSL",
               "Balance Sheet: Mortgage-Backed Securities ($ Mil)": "WSHOMCB",
               # other forms of lending
               "Balance Sheet: Repos ($ Mil)": "WORAL",
               "Balance Sheet: Central Bank Liquidity Swaps ($ Mil)" : "SWPT",
               "Balance Sheet: Direct Lending ($ Mil)" : "WLCFLL",
               # unamortized value of securities held (due to changes in interest rates)
               "Balance Sheet: Unamortized Security Premiums ($ Mil)": "WUPSHO",
               # Liabilities
               "Balance Sheet: Total Liabilities ($ Mil)" : "WLTLECL",
               "Balance Sheet: Federal Reserve Notes Outstanding ($ Mil)" : "WLFN",
               "Balance Sheet: Reverse Repos ($ Mil)": "WLRRAL",
               ### Major share of deposits 
               "Balance Sheet: Deposits from Dep. Institutions ($ Mil)":"WLODLL",
               "Balance Sheet: U.S. Treasury General Account ($ Mil)": "WDTGAL",
               "Balance Sheet: Other Deposits ($ Mil)": "WOTHLB",
               "Balance Sheet: All Deposits ($ Mil)": "WLDLCL",
               # Capital
               "Balance Sheet: Total Capital": "WCTCL",
               # Interest Rates
               "Unemployment Rate": "UNRATE",
               "Nominal GDP ($ Bil)":"GDP",
               "Real GDP ($ Bil)":"GDPC1",
               "GDP Deflator":"GDPDEF",
               "CPI":"CPIAUCSL",
               "Core PCE":"PCEPILFE",
               "Private Investment":"GPDI",
               "Base: Total ($ Mil)": "BOGMBASE",
               "Base: Currency in Circulation ($ Bil)": "WCURCIR",
               "1 Month Treasury Rate (%)": "DGS1MO",
               "3 Month Treasury Rate (%)": "DGS3MO",               
               "1 Year Treasury Rate (%)": "DGS1",
               "2 Year Treasury Rate (%)": "DGS2",
               "10 Year Treasury Rate (%)": "DGS10",
               "30 Year Treasury Rate (%)": "DGS30",               
               "Effective Federal Funds Rate (%)": "DFF",
               "Federal Funds Target Rate (Pre-crisis)":"DFEDTAR",
               "Federal Funds Upper Target":"DFEDTARU",
               "Federal Funds Lower Target":"DFEDTARL",
               "Interest on Reserves (%)": "IOER",
               "VIX": "VIXCLS",
                "5 Year Forward Rate": "T5YIFR"
               }

inflation_target = 2

unemployment_target = 4
# Select start and end dates
start = datetime.datetime(2000, 1, 1)
end = datetime.datetime.today()

## year variable automatically adjusts the numper of periods  
#   per year in light of data frequency
annual_div = {"Q":4,
             "W":52,
             "M":12}
### choose frequency
freq = "M"
### set periods per year
year = annual_div[freq]

In [2]:
# freq refers to data frequency. Choose "D", "W", "M", "Q", "A"
# a number may also be place in front of a letter. "2D" indicates
#       alternating days

if "data_gathered" not in locals():
    data = gather_data(data_codes, start, 
          end = end, freq = freq)
    data.fillna(0, inplace=True)
    for key in data.keys():
        data["Log " + key]= np.log(data[key])
    data_gathered = True

C:\Users\proma.gupta\Anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [4]:
data["Base: Currency in Circulation ($ Mil)"] = data["Base: Currency in Circulation ($ Bil)"].mul(1000)
data["Base: Currency in Circulation ($ Mil)"]

DATE
2002-12-31    6.764720e+05
2003-01-31    6.794114e+05
2003-02-28    6.798322e+05
2003-03-31    6.834130e+05
2003-04-30    6.870554e+05
                  ...     
2021-10-31    2.202444e+06
2021-11-30    2.211626e+06
2021-12-31    2.223944e+06
2022-01-31    2.233695e+06
2022-02-28    2.230436e+06
Freq: M, Name: Base: Currency in Circulation ($ Mil), Length: 231, dtype: float64

In [5]:
data["Currency in Circulation Growth Rate (%)"] = data["Base: Currency in Circulation ($ Mil)"].pct_change(year) * 100
data["Currency in Circulation Growth Rate (%)"]

DATE
2002-12-31         NaN
2003-01-31         NaN
2003-02-28         NaN
2003-03-31         NaN
2003-04-30         NaN
                ...   
2021-10-31    7.952279
2021-11-30    7.588562
2021-12-31    7.409584
2022-01-31    6.694934
2022-02-28    6.200318
Freq: M, Name: Currency in Circulation Growth Rate (%), Length: 231, dtype: float64

In [6]:
data["Currency in Circulation Growth Rate Diff (%)"] =  data["Currency in Circulation Growth Rate (%)"].diff(year)
data["Currency in Circulation Growth Rate Diff-in-Diff (%)"] =  data["Currency in Circulation Growth Rate Diff (%)"].diff(year)

In [7]:
data["Currency in Circulation Growth Rate Diff (%)"]

DATE
2002-12-31          NaN
2003-01-31          NaN
2003-02-28          NaN
2003-03-31          NaN
2003-04-30          NaN
                ...    
2021-10-31    -7.346889
2021-11-30    -7.534204
2021-12-31    -7.987665
2022-01-31    -9.678199
2022-02-28   -10.739284
Freq: M, Name: Currency in Circulation Growth Rate Diff (%), Length: 231, dtype: float64

In [8]:
data["Currency in Circulation Growth Rate Diff-in-Diff (%)"]

DATE
2002-12-31          NaN
2003-01-31          NaN
2003-02-28          NaN
2003-03-31          NaN
2003-04-30          NaN
                ...    
2021-10-31   -17.950979
2021-11-30   -17.720519
2021-12-31   -18.266402
2022-01-31   -20.830693
2022-02-28   -22.514329
Freq: M, Name: Currency in Circulation Growth Rate Diff-in-Diff (%), Length: 231, dtype: float64

In [9]:
data["Inflation (CPI)"] =  web.DataReader("CPIAUCSL", "fred", start, end).resample(freq).mean().pct_change(year).mul(100)
data["Inflation (PCE)"] = web.DataReader("PCEPILFE", "fred", start, end).resample(freq).mean().pct_change(year).mul(100)
data["Inflation (CPI)"]
data["Inflation (PCE)"]

DATE
2002-12-31    1.769002
2003-01-31    1.780060
2003-02-28    1.727237
2003-03-31    1.768547
2003-04-30    1.597828
                ...   
2021-10-31    4.156409
2021-11-30    4.659435
2021-12-31    4.850036
2022-01-31         NaN
2022-02-28         NaN
Freq: M, Name: Inflation (PCE), Length: 231, dtype: float64

In [10]:
data["Inflation (CPI)"]

DATE
2002-12-31    2.480271
2003-01-31    2.757456
2003-02-28    3.146067
2003-03-31    3.025210
2003-04-30    2.175125
                ...   
2021-10-31    6.236941
2021-11-30    6.828372
2021-12-31    7.096542
2022-01-31    7.525934
2022-02-28         NaN
Freq: M, Name: Inflation (CPI), Length: 231, dtype: float64

In [11]:
data["Unemployment Rate Diff"] = data["Unemployment Rate"].diff(year)
data["Unemployment Rate Diff-in-Diff"] = data["Unemployment Rate Diff"].diff(year)
data["Unemployment Rate Diff"] 
data["Unemployment Rate Diff-in-Diff"]

DATE
2002-12-31    NaN
2003-01-31    NaN
2003-02-28    NaN
2003-03-31    NaN
2003-04-30    NaN
             ... 
2021-10-31   -5.6
2021-11-30   -5.6
2021-12-31   -5.9
2022-01-31   -5.3
2022-02-28   -8.9
Freq: M, Name: Unemployment Rate Diff-in-Diff, Length: 231, dtype: float64

In [12]:
data["Unemployment Rate Diff"]

DATE
2002-12-31    NaN
2003-01-31    NaN
2003-02-28    NaN
2003-03-31    NaN
2003-04-30    NaN
             ... 
2021-10-31   -2.3
2021-11-30   -2.5
2021-12-31   -2.8
2022-01-31   -2.4
2022-02-28   -6.2
Freq: M, Name: Unemployment Rate Diff, Length: 231, dtype: float64

In [13]:
data["Inflation Loss"]= data["Inflation (PCE)"].sub(inflation_target)
data["Unemployment Loss"]= data["Unemployment Rate"].sub(unemployment_target)
data["Inflation Loss Sq"]= data["Inflation (PCE)"].sub(inflation_target).pow(2)
data["Inflation Loss Sq"][data["Inflation Loss"] < 0] = data["Inflation Loss Sq"].mul(-1)

data["Unemployment Loss Sq"]= data["Unemployment Rate"].sub(unemployment_target).pow(2)
data["Unemployment Loss Sq"][data["Unemployment Loss"] < 0] = data["Unemployment Loss Sq"].mul(-1)



data["Inflation Loss Diff"]= data["Inflation Loss"].diff(year)
data["Unemployment Loss Diff"]= data["Unemployment Loss"].diff(year)
data["Inflation Loss Sq Diff"]= data["Inflation Loss Sq"].diff(year)
data["Unemployment Loss Sq Diff"]= data["Unemployment Loss Sq"].diff(year)

data["Inflation Loss Diff-in-Diff"]= data["Inflation Loss Diff"].diff(year)
data["Unemployment Loss Diff-in-Diff"]= data["Unemployment Loss Diff"].diff(year)
data["Inflation Loss Sq Diff-in-Diff"]= data["Inflation Loss Sq Diff"].diff(year)
data["Unemployment Loss Sq Diff-in-Diff"]= data["Unemployment Loss Sq Diff"].diff(year)


data["Linear Loss"] = data["Inflation Loss"].sub(data["Unemployment Loss"])
data["Loss Function"] = data["Inflation Loss Sq"].sub(data["Unemployment Loss Sq"])
data["Linear Loss Diff"] = data["Linear Loss"].diff(year)
data["Loss Function Diff"] = data["Loss Function"].diff(year)
data["Linear Loss Diff-in-Diff"] = data["Linear Loss Diff"].diff(year)
data["Loss Function Diff-in-Diff"] = data["Loss Function Diff"].diff(year)

In [14]:
data["Inflation Loss"]

DATE
2002-12-31   -0.230998
2003-01-31   -0.219940
2003-02-28   -0.272763
2003-03-31   -0.231453
2003-04-30   -0.402172
                ...   
2021-10-31    2.156409
2021-11-30    2.659435
2021-12-31    2.850036
2022-01-31         NaN
2022-02-28         NaN
Freq: M, Name: Inflation Loss, Length: 231, dtype: float64

In [15]:
data["Unemployment Loss"]

DATE
2002-12-31    2.0
2003-01-31    1.8
2003-02-28    1.9
2003-03-31    1.9
2003-04-30    2.0
             ... 
2021-10-31    0.6
2021-11-30    0.2
2021-12-31   -0.1
2022-01-31    0.0
2022-02-28   -4.0
Freq: M, Name: Unemployment Loss, Length: 231, dtype: float64

In [16]:
data["Inflation Loss Sq"]

DATE
2002-12-31   -0.053360
2003-01-31   -0.048373
2003-02-28   -0.074400
2003-03-31   -0.053571
2003-04-30   -0.161742
                ...   
2021-10-31    4.650098
2021-11-30    7.072597
2021-12-31    8.122704
2022-01-31         NaN
2022-02-28         NaN
Freq: M, Name: Inflation Loss Sq, Length: 231, dtype: float64

In [17]:
data["Inflation Loss Sq"][data["Inflation Loss"] < 0] 

DATE
2002-12-31   -0.053360
2003-01-31   -0.048373
2003-02-28   -0.074400
2003-03-31   -0.053571
2003-04-30   -0.161742
                ...   
2020-11-30   -0.379513
2020-12-31   -0.270211
2021-01-31   -0.240884
2021-02-28   -0.262722
2021-03-31   -0.000703
Name: Inflation Loss Sq, Length: 160, dtype: float64

In [18]:
data["Unemployment Loss Sq"]

DATE
2002-12-31     4.00
2003-01-31     3.24
2003-02-28     3.61
2003-03-31     3.61
2003-04-30     4.00
              ...  
2021-10-31     0.36
2021-11-30     0.04
2021-12-31    -0.01
2022-01-31     0.00
2022-02-28   -16.00
Freq: M, Name: Unemployment Loss Sq, Length: 231, dtype: float64

In [19]:
data["Unemployment Loss Sq"][data["Unemployment Loss"] < 0]

DATE
2018-05-31    -0.04
2018-07-31    -0.04
2018-08-31    -0.04
2018-09-30    -0.09
2018-10-31    -0.04
2018-11-30    -0.04
2018-12-31    -0.01
2019-02-28    -0.04
2019-03-31    -0.04
2019-04-30    -0.16
2019-05-31    -0.16
2019-06-30    -0.16
2019-07-31    -0.09
2019-08-31    -0.09
2019-09-30    -0.25
2019-10-31    -0.16
2019-11-30    -0.16
2019-12-31    -0.16
2020-01-31    -0.25
2020-02-29    -0.25
2021-12-31    -0.01
2022-02-28   -16.00
Name: Unemployment Loss Sq, dtype: float64